# 6412B : Implémentation d'algorithme en recherche opérationnelle

# Phase 2 : arbre de recouvrement minimal

* Aurélie Boivin-Champeaux : matricule 2038926
* Adam Lebrigui : matricule 2037699

## Question 1 : Structure de données pour les composantes graphiques

### arbreRecouvrement.jl

Dans l'optique que nous devrons effectuer une recherche d'arbre de recouvrement minimal, nous avons choisi d'utiliser une structure ayant pour attributs :
* un dictionnaire ayant pour clé le noeud "enfant" et pour valeur le noeud "parent"
* Un tableau d'arête représentant les arêtes de l'arbre 

Le dictionnaire assure de pouvoir accéder rapidement à tous les noeuds en utilisant simplement le noeud "enfant" comme clé. Le tableau aurait pu avoir des complications dans les indices.  

La définition du type est accompagnée de fonctions "classiques" permettant à l'utilisateur de récupérer les attributs. La fonction récupérant le parent d'un noeud met une valeur par défaut à 0 si le noeud n'est pas trouvé dans le dictionnaire. 

In [ ]:
"""Type abstrait dont d'autres types de noeuds dériveront."""
abstract type AbstractArbre{T} end

"""Type représentant les parents de chaque noeud d'un graphe pour un arbre de recouvrement. """
mutable struct Arbre{T} <: AbstractArbre{T}
  name::String
  link::Dict{Node{T}, Node{T}}
  edges::Vector{Edge{T}}
end

"""Fonction retournant le dictionnaire contenant les noeuds parents de tous les noeuds"""
getParents(graphe::Arbre) = graphe.link

"""Fonction retournant le nom de l'objet arbre"""
getName(parent::AbstractArbre) = parent.name

"""Fonction récupérant les arêtes"""
getEdges(arbre::AbstractArbre) = arbre.edges

"""Fonction retournant le parent du noeud donné s'il existe"""
function getParent(parent::AbstractArbre{T}, noeud::AbstractNode{T}) where T
  return get(getParents(parent), noeud, 0)
end

"""Ajoute une arête au graphe."""
function add_edge!(arbre::AbstractArbre{T}, edge::Edge{T}) where T
  push!(arbre.edges, edge)
  arbre
end

### Fonctions supplémentaires

Nous avons ajouté en plus quelques fonctions qui sont utilisés pour l'algorithme de Kruskal. 
* changeParent! : cette fonction va modifier le noeud parent d'un noeud. Cette fonction va nous permettre de lier deux sous-arbres. 

* initArbre : cette fonction prenant un graphe en paramètre va initialiser un objet Arbre avec tous les noeuds ayant pour parents eux-mêmes.

* getRacine : cette fonction va renvoyer la racine d'un noeud donné en paramètre.

In [ ]:
"""Fonction changeant le parent d'un noeud"""
function changeParent!(tabParents::AbstractArbre, nodeChild::AbstractNode, nodeFather::AbstractNode)
  tabParents.link[nodeChild] = nodeFather
  return tabParents
end

"""Fonction initialisant un objet de type Arbre pour un graphe"""
function initArbre(graphe::AbstractGraph{T}) where T
  init = Dict{Node{typeNode(graphe)}, Node{typeNode(graphe)}}()
  edges = Edge{typeNode(graphe)}[]
  foret = Arbre(name(graphe), init, edges)
  for noeud in nodes(graphe)
    foret.link[noeud] = noeud
  end
  return foret
end


"""Fonction récupérant la racine du noeud précisé"""
function getRacine(arbre::AbstractArbre{T}, noeud::AbstractNode{T}) where T
  enfant = noeud
  parent = getParent(arbre, noeud)

  while enfant != parent
    enfant = parent
    parent = getParent(arbre, parent)
  end
  return enfant
end

## Question 2 : Implémentation de l'algorithme de Kruskal 

### kruskal.jl

La fonction prend en paramètre un graphe pour laquelle elle va initialiser un objet de type Arbre. Puis elle récupère les arêtes du graphe pour les trier par ordre croissant sur les poids. 

Elle applique ensuite une boucle sur toutes les arêtes pour les ajouter à l'arbre si elles relient deux ensembles différents. Les deux noeuds racines des noeuds de l'arête sont liés dans le dictionnaire de l'arbre. 

In [ ]:
"""fonction retournant un arbre de recouvrement minimal pour un graphe non orienté connexe"""
function algoKruskal(graph::AbstractGraph)
  # on crée un objet de type Arbre pour le graphe
  foret = initArbre(graph)

  # on crée un tableau avec toutes les arêtes du graphe triées par poids
  aretes = edges(graph)
  sort!(aretes, by = x -> x.weight)

  # pour chaque arête, on regarde si elle coupe un ensemble connexe cad si ses deux noeuds ont une racine différente
  for arete in aretes
    # si oui, on relie les deux ensembles connexes par leurs racines
    if getRacine(foret, getNode1(arete)) != getRacine(foret, getNode2(arete))
      changeParent!(foret, getRacine(foret, getNode1(arete)), getRacine(foret, getNode2(arete)))
      # On ajoute l'arête à l'arbre
      add_edge!(foret, arete)
    end
    #  sinon on passe à l'arête suivante
  end
  return foret

end

## Question 3 : Tests unitaires

### tests.jl

Nous avons créé un nouveau fichier pour effectuer tous nos tests unitaires au long du codage. Ils se basent sur un graphe composé de 3 noeuds et 3 arêtes de poids différents. 

In [ ]:

# test node.jl
node1 = Node(1,2)
@test name(node1) == 1
@test data(node1) == 2
node2 = Node(2,1)
node3 = Node(3,1)

# test edge.jl
edge1 = Edge(node1, node2, 4)
@test getNode1(edge1) == node1
@test getNode2(edge1) == node2
@test weight(edge1) == 4
edge2 = Edge(node2, node3, 2)

# test graph.jl
graphe = Graph("test", [node1, node2],[edge1] )
@test name(graphe) == "test"
@test nb_nodes(graphe) == 2
@test nb_edges(graphe) == 1
@test typeNode(graphe) == Int64
@test nodes(graphe) == [node1, node2]
@test edges(graphe) == [edge1]
add_node!(graphe,  node3)
@test nodes(graphe) == [node1, node2, node3]
add_edge!(graphe, edge2)
@test edges(graphe) == [edge1, edge2]


# test recouvrement.jl
foret = initArbre(graphe)
@test getName(foret) == name(graphe)
@test getParents(foret) == Dict(node3 => node3, node1 => node1, node2 => node2)
@test getEdges(foret) == Edge{Int64}[]
@test getParent(foret, node1) == node1
@test getRacine(foret, node1) == node1
changeParent!(foret, node1, node2)
@test getParent(foret, node1) == node2
changeParent!(foret, node2, node3)
@test getRacine(foret, node1) == node3

# test algoKruskal
edge3 = Edge(node1, node3, 1)
add_edge!(graphe, edge3)
# test du bon fonctionnement de l'algo
arbre = algoKruskal(graphe)
@test getRacine(arbre, node3) == node3
@test getRacine(arbre, node2) == node3
@test getRacine(arbre, node1) == node3
@test getEdges(arbre) == [edge3, edge2]

## Question 4 : Application sur des instances TSP

### main.jl

La fonction prend en argument un graphe et renvoie un objet de type Arbre contenant l'arbre de recouvrement minimal

In [ ]:
function main(graphe::AbstractGraph)

    arbre = algoKruskal(graphe)

end

### phase2Exemples.jl

L'algorithme est testé sur l'exemple du cours et sur 3 fichiers différents. Chacun représente un type de matrice différent (lower_diagonal_row, upper_row, full_matrix)

In [ ]:
function test()

    # Test sur l'arbre vu en cours
    # Construction des noeuds
    node1 = Node(1, "a")
    node2 = Node(2, "b")
    node3 = Node(3, "C")
    node4 = Node(4, "d")
    node5 = Node(5, "e")
    node6 = Node(6, "f")
    node7 = Node(7, "g")
    node8 = Node(8, "h")
    node9 = Node(9, "i")
    nodes = [node1, node2, node3, node4, node5, node6, node7, node8, node9]

    # Construction des arêtes
    edge1 = Edge(node1, node2, 4)
    edge2 = Edge(node1, node8, 8)
    edge3 = Edge(node2, node3, 8)
    edge4 = Edge(node2, node8, 11)
    edge5 = Edge(node3, node9, 2)
    edge6 = Edge(node3, node6, 4)
    edge7 = Edge(node3, node4, 7)
    edge8 = Edge(node4, node5, 9)
    edge9 = Edge(node4, node6, 14)
    edge10 = Edge(node5, node6, 10)
    edge11 = Edge(node7, node9, 6)
    edge12 = Edge(node7, node8, 1)
    edge13 = Edge(node8, node9, 7)
    edge14 = Edge(node6, node7, 2)
    edges = [edge1, edge2, edge3, edge4, edge5, edge6, edge7, edge8, edge9, edge10, edge11, edge12, edge13, edge14]

    grapheCours = Graph("cours", nodes, edges)
    arbreCours = main(grapheCours)
    # vérification sur les arêtes de l'arbre de recouvrement minimal
    show(arbreCours.edges)


    file_name = "hk48.tsp"
    # on construit le graphe à partir d'une instance TSP
    graphe1 = construct_graph(file_name, "test")

    # on construit l'arbre de recouvrement minimal grâce à l'algo de Kruskal
    arbre1 = main(graphe1)
    # show(arbre1.edges)

    # test 2
    graphe2 = construct_graph("bayg29.tsp", "test2")
    arbre2 = main(graphe2)
    # show(arbre2.edges)

    # test3
    graphe3 = construct_graph("swiss42.tsp", "test4")
    arbre3 = main(graphe3)
    # show(arbre3.edges)
end

test()
